In [3]:
import pandas as pd
import os

df_train = pd.read_parquet(os.path.join("data", "postprocessed", "training", "TRAIN_CLEAN_v6.parquet"))
df_rank = pd.read_parquet(os.path.join("data", "postprocessed", "training", "RANK_CLEAN_v6.parquet"))
df_final = pd.read_parquet(os.path.join("data", "postprocessed", "training", "FINAL_CLEAN_v6.parquet"))


print(df_train.isnull().sum())
print(df_rank.isnull().sum())
print(df_final.isnull().sum())


flight_id                        0
typecode                         0
source                           0
fuel_kg                          0
dev_start_s                      0
dev_end_s                        0
wake                             0
engine_type                      0
engine_number                    0
segment_start                    0
segment_end                      0
segment_ground_like              0
segment_state                    0
origin_icao_type                 0
dest_icao_type                   0
origin_icao                      0
destination_icao                 0
origin_apt_longitude             0
origin_apt_latitude              0
origin_apt_elevation             0
dest_apt_longitude               0
dest_apt_latitude                0
dest_apt_elevation               0
total_flight_duration_s          0
takeoff                          0
landing                          0
segment_duration_s               0
segment_start_time_s             0
segment_start_percen

In [10]:
# 1) Create datasets that contain ONLY rows with at least one null
df_train_nulls = df_train[df_train.isnull().any(axis=1)].copy()
df_rank_nulls  = df_rank[df_rank.isnull().any(axis=1)].copy()
df_final_nulls = df_final[df_final.isnull().any(axis=1)].copy()

In [21]:
import pandas as pd
import os

df_train = pd.read_parquet(os.path.join("data", "postprocessed", "training", "TRAIN_CLEAN_v6.parquet"))
df_rank  = pd.read_parquet(os.path.join("data", "postprocessed", "training", "RANK_CLEAN_v6.parquet"))
df_final = pd.read_parquet(os.path.join("data", "postprocessed", "training", "FINAL_CLEAN_v6.parquet"))

# --- TRAIN: normal null-row selection (consider all cols) ---
df_train_nulls = df_train[df_train.isnull().any(axis=1)].copy()

# --- RANK: ignore fuel_kg when checking for nulls ---
rank_cols_no_fuel = [c for c in df_rank.columns if c != "fuel_kg"]
df_rank_nulls = df_rank[df_rank[rank_cols_no_fuel].isnull().any(axis=1)].copy()

# --- FINAL: ignore fuel_kg when checking for nulls ---
final_cols_no_fuel = [c for c in df_final.columns if c != "fuel_kg"]
df_final_nulls = df_final[df_final[final_cols_no_fuel].isnull().any(axis=1)].copy()

print("Train null rows:", df_train_nulls.shape)
print("Rank null rows (ignoring fuel_kg):", df_rank_nulls.shape)
print("Final null rows (ignoring fuel_kg):", df_final_nulls.shape)

# Optional: save as datasets
out_dir = os.path.join("data", "postprocessed", "null_analysis")
os.makedirs(out_dir, exist_ok=True)

# df_train_nulls.to_parquet(os.path.join(out_dir, "TRAIN_NULL_ROWS.parquet"))
# df_rank_nulls.to_parquet(os.path.join(out_dir, "RANK_NULL_ROWS.parquet"))
# df_final_nulls.to_parquet(os.path.join(out_dir, "FINAL_NULL_ROWS.parquet"))


Train null rows: (49055, 55)
Rank null rows (ignoring fuel_kg): (7553, 55)
Final null rows (ignoring fuel_kg): (19617, 55)


In [30]:
print(df_train_nulls["source"].value_counts())
print(df_rank_nulls["source"].value_counts())
print(df_final_nulls["source"].value_counts())

df = pd.read_parquet(os.path.join("data", "flights_train", "prc770852254.parquet"))

print(df.columns)


source
acars    37389
adsb     11666
Name: count, dtype: int64
source
acars    6789
adsb      764
Name: count, dtype: int64
source
acars    17441
adsb      2176
Name: count, dtype: int64
Index(['timestamp', 'flight_id', 'typecode', 'latitude', 'longitude',
       'altitude', 'groundspeed', 'track', 'vertical_rate', 'mach', 'TAS',
       'CAS', 'source'],
      dtype='object')


In [28]:
# 1) Split by source
df_adsb  = df_train_nulls[df_train_nulls["source"] == "adsb"].copy()
df_acars = df_train_nulls[df_train_nulls["source"] == "acars"].copy()

print(len(df_adsb), "ADSB rows")
print(len(df_acars), "ACARS rows")

# 2) Count nulls per column for ADSB
nulls_adsb = df_adsb.isnull().sum().sort_values(ascending=False)
print("\nNull counts in ADSB:")
print(nulls_adsb[nulls_adsb > 0])

# 3) Count nulls per column for ACARS
nulls_acars = df_acars.isnull().sum().sort_values(ascending=False)
print("\nNull counts in ACARS:")
print(nulls_acars[nulls_acars > 0])

null_compare = pd.DataFrame({
    "adsb_nulls": nulls_adsb,
    "acars_nulls": nulls_acars
}).fillna(0).astype(int)

print(null_compare)


11666 ADSB rows
37389 ACARS rows

Null counts in ADSB:
mach_delta             11666
mach_mean              11413
groundspeed_delta          5
track_delta                5
vertical_rate_delta        5
track_mean                 1
groundspeed_mean           1
vertical_rate_mean         1
dtype: int64

Null counts in ACARS:
mach_delta               33401
mach_mean                33091
track_delta              13392
vertical_rate_delta      13392
track_mean               12796
vertical_rate_mean       12796
groundspeed_delta         1388
groundspeed_mean           352
dist_to_dest_delta_nm      331
dist_to_dest_start_nm      331
latitude_delta             325
longitude_delta            325
dist_to_dest_end_nm        290
latitude_mean              285
longitude_mean             285
altitude_delta              66
altitude_mean               55
dtype: int64
                           adsb_nulls  acars_nulls
altitude_delta                      0           66
altitude_mean                      

In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype

def compare_parquet_all_columns(
    old_path: str,
    new_path: str,
    rtol: float = 1e-8,
    atol: float = 1e-10,
):
    """
    Compare two parquet files column by column and row by row.

    Assumptions:
    - Same logical rows in the same order (same index / same row count).
    - New file may have extra columns, old file may have columns that disappeared.

    Returns:
        {
            "columns_only_in_old": [list of col names],
            "columns_only_in_new": [list of col names],
            "value_differences": DataFrame with:
                ['row_index', 'column', 'old_value', 'new_value']
        }
    """
    old_df = pd.read_parquet(old_path)
    new_df = pd.read_parquet(new_path)

    # Basic row check (you can drop this if rows are known to differ)
    if len(old_df) != len(new_df):
        raise ValueError(f"Row count mismatch: old={len(old_df)}, new={len(new_df)}")

    # Column sets
    old_cols = set(old_df.columns)
    new_cols = set(new_df.columns)

    columns_only_in_old = sorted(old_cols - new_cols)
    columns_only_in_new = sorted(new_cols - old_cols)
    common_cols = sorted(old_cols & new_cols)

    # Compare shared columns
    old_common = old_df[common_cols]
    new_common = new_df[common_cols]

    diff_mask = pd.DataFrame(False, index=old_common.index, columns=common_cols)

    for col in common_cols:
        s_old = old_common[col]
        s_new = new_common[col]

        if is_numeric_dtype(s_old) and is_numeric_dtype(s_new):
            a = s_old.to_numpy()
            b = s_new.to_numpy()
            # True where values are effectively equal
            close = np.isclose(a, b, rtol=rtol, atol=atol, equal_nan=True)
            diff_mask[col] = ~close
        else:
            # Non-numeric: exact match, but NaN == NaN is treated as equal
            both_nan = s_old.isna() & s_new.isna()
            eq = (s_old == s_new) | both_nan
            diff_mask[col] = ~eq

    if not diff_mask.any().any():
        value_differences = pd.DataFrame(
            columns=["row_index", "column", "old_value", "new_value"]
        )
    else:
        diffs = []
        for col in common_cols:
            changed_rows = diff_mask[col]
            if changed_rows.any():
                diff_data = pd.DataFrame({
                    "row_index": old_common.index[changed_rows],
                    "column": col,
                    "old_value": old_common.loc[changed_rows, col].values,
                    "new_value": new_common.loc[changed_rows, col].values,
                })
                diffs.append(diff_data)
        value_differences = pd.concat(diffs, ignore_index=True)

    return {
        "columns_only_in_old": columns_only_in_old,
        "columns_only_in_new": columns_only_in_new,
        "value_differences": value_differences,
    }


In [7]:
import os
new = os.path.join("data","postprocessed", "training", "TRAIN_CLEAN_v2.parquet")
old = os.path.join("data","postprocessed", "training", "TRAIN_CLEAN_v2_1.parquet")

result = compare_parquet_all_columns(old, new)

print("Only in old:", result["columns_only_in_old"])
print("Only in new:", result["columns_only_in_new"])

diffs = result["value_differences"]
print("Number of differing cells:", len(diffs))
print(diffs.head())

Only in old: []
Only in new: ['engine_number', 'engine_type', 'flightphase_dominant', 'landing', 'segment_cl', 'segment_cr', 'segment_de', 'segment_gnd', 'segment_lvl', 'segment_na', 'segment_start_percentage', 'segment_start_time_s', 'takeoff', 'total_flight_duration_s', 'wake']
Number of differing cells: 0
Empty DataFrame
Columns: [row_index, column, old_value, new_value]
Index: []


In [19]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

def plot_flight(df, flight_id=None):
    """
    df: single-flight dataframe containing timestamp + trajectory variables
    """

    # Sort by timestamp
    df = df.sort_values("timestamp").reset_index(drop=True)

    # Convert to datetime if needed
    if not pd.api.types.is_datetime64_any_dtype(df["timestamp"]):
        df["timestamp"] = pd.to_datetime(df["timestamp"])

    title_prefix = f"Flight {flight_id}" if flight_id else "Flight"

    # ----------------------
    # 1. 3D TRAJECTORY PLOT
    # ----------------------
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot(111, projection="3d")
    ax.plot(df["longitude"], df["latitude"], df["altitude"])
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_zlabel("Altitude [ft]")
    ax.set_title(f"{title_prefix} – 3D Flight Trajectory")
    plt.show()

    # ----------------------
    # 2. MAP VIEW (LAT-LON)
    # ----------------------
    plt.figure(figsize=(9,6))
    plt.plot(df["longitude"], df["latitude"], linewidth=2)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title(f"{title_prefix} – Ground Track")
    plt.grid(True)
    plt.show()

    # ----------------------
    # 3. ALTITUDE OVER TIME
    # ----------------------
    plt.figure(figsize=(9,4))
    plt.plot(df["timestamp"], df["altitude"])
    plt.xlabel("Time")
    plt.ylabel("Altitude [ft]")
    plt.title(f"{title_prefix} – Altitude Profile")
    plt.grid(True)
    plt.show()

    # ----------------------
    # 4. GROUNDSPEED OVER TIME
    # ----------------------
    if "groundspeed" in df:
        plt.figure(figsize=(9,4))
        plt.plot(df["timestamp"], df["groundspeed"])
        plt.xlabel("Time")
        plt.ylabel("Groundspeed [kt]")
        plt.title(f"{title_prefix} – Groundspeed Profile")
        plt.grid(True)
        plt.show()

    # ----------------------
    # 5. VERTICAL RATE OVER TIME
    # ----------------------
    if "vertical_rate" in df:
        plt.figure(figsize=(9,4))
        plt.plot(df["timestamp"], df["vertical_rate"])
        plt.xlabel("Time")
        plt.ylabel("Vertical Rate [ft/min]")
        plt.title(f"{title_prefix} – Vertical Rate Profile")
        plt.grid(True)
        plt.show()

    # ----------------------
    # 6. TRACK (HEADING) OVER TIME
    # ----------------------
    if "track" in df:
        plt.figure(figsize=(9,4))
        plt.plot(df["timestamp"], df["track"])
        plt.xlabel("Time")
        plt.ylabel("Track Angle [deg]")
        plt.title(f"{title_prefix} – Track (Heading) Profile")
        plt.grid(True)
        plt.show()


In [ ]:
# Suppose df_flight contains all points for one flight_id
df_flight = df_train[df_train["flight_id"] == "AA123"].copy()

plot_flight(df_flight, flight_id="AA123")
